In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
###Linear Regression###

#Creating known weights and biases
weight = torch.randn(1)
bias = torch.randn_like(weight)

start = 1
end = 2
step = 0.01

x = torch.arange(start,end,step)
y = x * weight + bias

train_len = round(0.8 * len(x))
X_train, X_test = x[:train_len], x[train_len:]
y_train, y_test = y[:train_len], y[train_len:]

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1,dtype=torch.float),requires_grad=True)
        self.bias = nn.Parameter(torch.randn(1,dtype=torch.float),requires_grad=True)
    
    def forward(self,x):
        return x * self.weight + self.bias

linearmodel_0 = LinearRegressionModel()

def plot_predictions(train_data = X_train,
                     train_labels = y_train,
                     val_data = X_test,
                     val_labels = y_test,
                     predictions = None):
    plt.figure(figsize=(10,7))

    plt.scatter(train_data,train_labels,c='b',s=4,label="Train data")
    plt.scatter(val_data,val_labels,c='g',s=4,label="Test data")

    if predictions is not None:
        plt.scatter(val_data,predictions,c='r',s=4,label="Predictions")
    plt.legend()
    plt.show()


loss_fn = nn.MSELoss()
optimiser = torch.optim.SGD(params=linearmodel_0.parameters(),lr=1e-1)
epochs = 1500

for epoch in range(epochs):
    linearmodel_0.train()
    preds = linearmodel_0(X_train)
    loss = loss_fn(preds, y_train)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if epoch%100==0:
        print(f"Model parameters : {linearmodel_0.state_dict()}")

linearmodel_0.eval()
preds = linearmodel_0(X_test).detach().numpy()
print(f"Model parameters : {linearmodel_0.state_dict()} | \nActual parameters : weight -> {weight}, bias -> {bias}")
plot_predictions(predictions=preds)

In [ ]:
import torch
import torch.nn as nn
import sklearn
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

In [135]:
###Binary Classification###

X,y = make_circles(1000,noise=0.02,random_state=42,)

X,y = torch.from_numpy(X).type(torch.float), torch.from_numpy(y).type(torch.float)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#shape changes during training : [800,2] -> [800,hidden features] -> [800, hidedn features] -> [800, output features] -> sigmoid and round -> [800]

INPUT_FEATURES = X_train.shape[1]
HIDDEN_FEATURES = 10
OUTPUT_FEATURES = 1

class BinaryModel(nn.Module):
    def __init__(self, input_features,hidden_features,output_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_features,hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features,hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features,output_features), #shape = [800,output_features]
            nn.Sigmoid(),
        )
    
    def forward(self,x):
        return self.model(x)

def accuracy_fn(y_true,y_preds):
    x = torch.eq(y_true,y_preds).sum().item()
    return (x/len(y_true)) * 100

binarymodel0 = BinaryModel(INPUT_FEATURES,HIDDEN_FEATURES,OUTPUT_FEATURES)

loss_fn = nn.BCEWithLogitsLoss()
optimiser = torch.optim.SGD(params=binarymodel0.parameters(),lr=1e-1)

epochs = 10000
for epoch in range(epochs):
    binarymodel0.train()
    logits = binarymodel0(X_train).squeeze()
    preds = torch.round(logits)
    loss = loss_fn(logits,y_train)
    acc = accuracy_fn(y_train,preds)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if epoch % 100 == 0:
        print(f"Train Loss : {loss:.3f} | Train acc : {acc:.2f}%")
    if epoch%1000 == 0:
        print(preds==y_train)

# binarymodel0.eval()
# preds = binarymodel0(X)
# preds.sigmoid().round() == y_train


Train Loss : 0.720 | Train acc : 50.00%
tensor([False,  True,  True,  True, False,  True, False, False,  True,  True,
        False,  True,  True,  True, False,  True,  True,  True,  True, False,
        False, False,  True, False, False, False,  True,  True,  True,  True,
         True,  True,  True, False,  True,  True, False, False, False, False,
        False, False,  True, False,  True, False,  True,  True,  True,  True,
         True, False,  True, False, False, False,  True, False, False, False,
         True,  True,  True, False,  True,  True, False,  True,  True,  True,
        False, False, False,  True,  True,  True,  True,  True,  True,  True,
        False, False,  True, False,  True, False,  True, False,  True, False,
        False, False,  True,  True,  True, False,  True, False, False,  True,
         True,  True,  True,  True,  True, False,  True,  True,  True,  True,
        False, False,  True,  True, False,  True, False, False,  True, False,
         True, False,  T

In [150]:
import torch 
import torch.nn as nn
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from torchmetrics.classification import MulticlassAccuracy

In [157]:
###Multi Class Classification###
NUM_FEATURES = 2
NUM_CENTERS = 5
HIDDEN_FEATURES = 10

X,y = make_blobs(n_samples=1000,n_features=NUM_FEATURES,centers=NUM_CENTERS,random_state=42)
X,y = torch.from_numpy(X).type(torch.float), torch.from_numpy(y).type(torch.LongTensor)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

class MultiClassModel(nn.Module):
    def __init__(self,input_features,hidden_features,output_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_features,hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features,hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features,output_features),
            nn.Sigmoid(),
        )
    
    def forward(self,x):
        return self.model(x)

multiclassmodel0 = MultiClassModel(NUM_FEATURES,HIDDEN_FEATURES,NUM_CENTERS)

loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(params=multiclassmodel0.parameters(),lr=1e-1)
accuracy = MulticlassAccuracy(num_classes=NUM_CENTERS)

epochs = 10000
for epoch in range(epochs):
    multiclassmodel0.train()
    logits = multiclassmodel0(X_train)
    preds = torch.softmax(logits,dim=1).argmax(dim=1).squeeze()
    loss = loss_fn(logits,y_train)
    acc = accuracy(y_train,preds)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if epoch%50 == 0:
        print(f"Train loss : {loss:.3f} | Train acc : {acc:.3f}")

Train loss : 1.602 | Train acc : 0.049
Train loss : 1.340 | Train acc : 0.460
Train loss : 1.166 | Train acc : 0.621
Train loss : 1.077 | Train acc : 0.690
Train loss : 1.031 | Train acc : 0.692
Train loss : 1.014 | Train acc : 0.692
Train loss : 1.003 | Train acc : 0.692
Train loss : 0.992 | Train acc : 0.692
Train loss : 0.983 | Train acc : 0.692
Train loss : 0.978 | Train acc : 0.692
Train loss : 0.974 | Train acc : 0.692
Train loss : 0.972 | Train acc : 0.692
Train loss : 0.971 | Train acc : 0.692
Train loss : 0.970 | Train acc : 0.692
Train loss : 0.969 | Train acc : 0.692
Train loss : 0.969 | Train acc : 0.692
Train loss : 0.968 | Train acc : 0.692
Train loss : 0.968 | Train acc : 0.692
Train loss : 0.967 | Train acc : 0.692
Train loss : 0.967 | Train acc : 0.692
Train loss : 0.967 | Train acc : 0.692
Train loss : 0.967 | Train acc : 0.692
Train loss : 0.966 | Train acc : 0.692
Train loss : 0.966 | Train acc : 0.692
Train loss : 0.966 | Train acc : 0.692
Train loss : 0.966 | Trai

In [ ]:
()